In [19]:
import numpy as np
import pandas as pd
import os

In [2]:
initial_str = "0101101101110111"

sistring_list = []

for i in range(len(initial_str)):
    sistring = initial_str[i:]
    sistring_list.append(sistring)
    print(sistring)

    if i == 8:
        break

0101101101110111
101101101110111
01101101110111
1101101110111
101101110111
01101110111
1101110111
101110111
01110111


In [3]:
def get_sistring_list(initial_str: str, num_of_sistrings:int = 9) -> list:
    sistring_list = []
    for i in range(len(initial_str)):
        sistring = initial_str[i:]
        sistring_list.append(sistring)
        if i == num_of_sistrings - 1:
            break
        
    return sistring_list

In [4]:
def get_unique_prefixes(sistring_list: list) -> list:

    # Find the longest common prefix
    prefix = os.path.commonprefix(sistring_list)
    unique_prefixes = []
    # Print the unique prefix for each string
    for string in sistring_list:
        if string.startswith(prefix):
            # Remove the prefix from the current string to get the unique prefix
            unique_prefix = string.replace(prefix, '', 1)
            unique_prefixes.append(unique_prefix)
    
    return unique_prefixes            

In [5]:
initial_str = "0101101101110111"
sistrings = get_sistring_list(initial_str, 9)
sistrings

['0101101101110111',
 '101101101110111',
 '01101101110111',
 '1101101110111',
 '101101110111',
 '01101110111',
 '1101110111',
 '101110111',
 '01110111']

In [6]:
get_unique_prefixes(sistrings)

['0101101101110111',
 '101101101110111',
 '01101101110111',
 '1101101110111',
 '101101110111',
 '01101110111',
 '1101110111',
 '101110111',
 '01110111']

In [7]:
from pygtrie import Trie

def unique_prefixes(strings):
    trie = Trie()
    prefixes = {}
    for s in strings:
        for i in range(len(s)):
            prefix = s[:i+1]
            if prefix in prefixes:
                del prefixes[prefix]
            elif prefix not in trie:
                trie[prefix] = True
                prefixes[prefix] = s
    return list(prefixes.keys())

print(unique_prefixes(sistrings))


['010', '0101', '01011', '010110', '0101101', '01011011', '010110110', '0101101101', '01011011011', '010110110111', '0101101101110', '01011011011101', '010110110111011', '0101101101110111', '10110110', '101101101', '1011011011', '10110110111', '101101101110', '1011011011101', '10110110111011', '101101101110111', '0110110', '01101101', '011011011', '0110110111', '01101101110', '011011011101', '0110110111011', '01101101110111', '110110', '1101101', '11011011', '110110111', '1101101110', '11011011101', '110110111011', '1101101110111', '10110111', '101101110', '1011011101', '10110111011', '101101110111', '0110111', '01101110', '011011101', '0110111011', '01101110111', '110111', '1101110', '11011101', '110111011', '1101110111', '10111', '101110', '1011101', '10111011', '101110111', '0111', '01110', '011101', '0111011', '01110111']


In [8]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False
        self.prefix_count = 0

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
            node.prefix_count += 1
        node.is_end_of_word = True

    def find_unique_prefix(self, word):
        node = self.root
        prefix = ""
        for char in word:
            node = node.children[char]
            prefix += char
            if node.prefix_count == 1:
                return prefix
        return prefix

trie = Trie()
for string in sistrings:
    trie.insert(string)

unique_prefixes = []

for string in sistrings:
    
    unique_prefix = trie.find_unique_prefix(string)
    unique_prefixes.append(unique_prefix)
    print(unique_prefix)

010
10110110
0110110
110110
10110111
0110111
110111
10111
0111


In [9]:
import pydot

class Node:
    def __init__(self, char):
        self.char = char
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)

    def get_child(self, char):
        for child in self.children:
            if child.char == char:
                return child
        return None

class PAT_Trie:
    def __init__(self):
        self.root = Node('')

    def insert(self, word):
        current_node = self.root
        for char in word:
            child = current_node.get_child(char)
            if child:
                current_node = child
            else:
                new_node = Node(char)
                current_node.add_child(new_node)
                current_node = new_node

    def draw(self, filename):
        graph = pydot.Dot(graph_type='graph')

        def add_node(node, parent):
            label = node.char
            if not label:
                label = "root"
            graph_node = pydot.Node(str(node), label=label)
            graph.add_node(graph_node)
            if parent is not None:
                graph_edge = pydot.Edge(str(parent), str(node))
                graph.add_edge(graph_edge)
            for child in node.children:
                add_node(child, node)

        add_node(self.root, None)
        graph.write_png(filename)

# example usage


trie = PAT_Trie()
for s in sistrings:
    trie.insert(s)

trie.draw('pat_trie.png')


In [17]:
from graphviz import Digraph

def draw_pat_trie(strings):
    # Initialize trie graph
    trie = Digraph()

    # Add root node
    trie.node("root", label="", shape="circle")

    # Loop over strings
    for s in strings:
        # Add string to trie
        current_node = "root"
        prefix = ""
        for i, c in enumerate(s):
            prefix += c
            if prefix not in [edge.attr['label'] for edge in trie.edges(current_node)]:
                # Add new node and edge
                trie.node(prefix, shape="circle")
                trie.edge(current_node, prefix, label=prefix)
            current_node = prefix

    # Set graph attributes
    trie.attr(rankdir="LR")
    trie.attr("node", shape="circle")
    trie.attr("edge", arrowhead="none")

    # Render graph
    trie.view()

# Example usage
draw_pat_trie(sistrings)


ValueError: not enough values to unpack (expected 2, got 1)

In [11]:
import pydot

class TrieNode:
    def __init__(self, char: str):
        self.char = char
        self.children = []
        self.word_finished = False

def add(root, word: str):
    node = root
    for char in word:
        found_in_child = False
        for child in node.children:
            if child.char == char:
                node = child
                found_in_child = True
                break
        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            node = new_node
    node.word_finished = True

def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)

def visit(node, parent=None):
    for child in node.children:
        if parent:
            draw(parent.char, child.char)
        visit(child, child)

root = TrieNode('*')

for string in sistrings:
    add(root, string)

graph = pydot.Dot(graph_type='digraph')
visit(root)

graph.write_png('pat_trie.png')


# Q2

In [37]:
import collections

In [49]:
# Define the documents
corpus = ["lemon, banana, banana, plum, plum, pear",
            "pear, banana, banana, plum, banana, plum",
            "lemon, banana, lemon, plum, lemon",
            "lemon, banana, lemon, lemon, plum, plum, lemon"]


documents = [d.split(', ') for d in corpus]
corpus_joined = ", ".join(corpus).split(', ')
total_words = len(corpus_joined)

# Calculate the probabilities of each word occurring
word_counts = collections.Counter(corpus_joined)


print(">>> Probabilities if in the category fruit:")

for key, val in word_counts.items():
    print(f'Probability of the word {key}: ({val}+1) / ({total_words}+{len(corpus)})')


>>> Probabilities if in the category fruit:
Probability of the word lemon: (8+1) / (24+4)
Probability of the word banana: (7+1) / (24+4)
Probability of the word plum: (7+1) / (24+4)
Probability of the word pear: (2+1) / (24+4)


In [50]:
# Define the documents
corpus = ["banana, lemon, pear, banana",
            "lemon, banana, banana, lemon",
            "banana, lemon, lemon, lemon",
            "lemon, lemon"]


documents = [d.split(', ') for d in corpus]
corpus_joined = ", ".join(corpus).split(', ')
total_words = len(corpus_joined)

# Calculate the probabilities of each word occurring
word_counts = collections.Counter(corpus_joined)


print(">>> Probabilities if in the category not fruit:")

for key, val in word_counts.items():
    print(f'Probability of the word {key}: ({val}+1) / ({total_words}+{len(corpus)})')


>>> Probabilities if in the category not fruit:
Probability of the word banana: (5+1) / (14+4)
Probability of the word lemon: (8+1) / (14+4)
Probability of the word pear: (1+1) / (14+4)


In [45]:
unique_in_documents = np.hstack([np.unique(d.split(', ')) for d in corpus])

total_words = len(unique_in_documents)

# Calculate the probabilities of each word occurring
word_counts = collections.Counter(unique_in_documents)
proba_occurrence_in_doc = {key: f"{val} / {len(corpus)}" for key, val in word_counts.items()}


for key, val in proba_occurrence_in_doc.items():
    print(f'Probability of the word {key} occurring in a document: {val}'   )

Probability of the word banana occurring in a document: 4 / 4
Probability of the word lemon occurring in a document: 3 / 4
Probability of the word pear occurring in a document: 2 / 4
Probability of the word plum occurring in a document: 4 / 4


In [47]:
# Calculate the probabilities of each word occurring
word_counts = collections.Counter(corpus_joined)
word_proba = {key: val / total_words for key, val in word_counts.items()}


words_idf = {key: -np.log2(val) for key, val in word_proba.items()}

for key, val in words_idf.items():
    print(f'Information value (IDF) of the word {key}: {val }')
    
avg_info = 0

for key, val in words_idf.items():
    avg_info += val * word_proba[key]
    
print("Average information value of the whole corpus:", avg_info)

Information value (IDF) of the word lemon: 1.5849625007211563
Information value (IDF) of the word banana: 1.777607578663552
Information value (IDF) of the word plum: 1.777607578663552
Information value (IDF) of the word pear: 3.584962500721156
Average information value of the whole corpus: 1.8640054628542204


# Q3

In [21]:
def generate_shingles(doc: str, n: int=3):
    shingles = doc.replace("W", "").split()
    comb_s = [int("".join(shingles[i: i+n])) for i in range(len(shingles)-(n-1))]
    return comb_s

In [23]:
doc1 = "W3	W6	W3	W6	W7	W8	W9	W1	W4	W7	W3	W2	W1	W9"
doc2 = "W4	W7	W3	W6	W7	W8	W9	W1	W4	W7	W5	W3	W8	W9"

comb_s1 = generate_shingles(doc1)
comb_s2 = generate_shingles(doc2)

data = np.array([comb_s1, comb_s2]).T
df = pd.DataFrame(data, columns=["doc1", "doc2"])
df

,doc1,doc2
0,363,473
1,636,736
2,367,367
3,678,678
4,789,789
5,891,891
6,914,914
7,147,147
8,473,475
9,732,753


In [28]:
def calculate_similarity(doc1, doc2, num_lowest_shingles: int=None):
    
    if num_lowest_shingles:
        comb_s2 = generate_shingles(doc2)[:num_lowest_shingles]
        comb_s1 = generate_shingles(doc1)[:num_lowest_shingles]
    else:
        comb_s2 = generate_shingles(doc2)
        comb_s1 = generate_shingles(doc1)
        
    return f"{len(set(comb_s1).intersection(set(comb_s2)))} / {len(set(comb_s1).union(set(comb_s2)))}"

doc1 = "W3	W6	W3	W6	W7	W8	W9	W1	W4	W7	W3	W2	W1	W9"
doc2 = "W4	W7	W3	W6	W7	W8	W9	W1	W4	W7	W5	W3	W8	W9"


resemblence = calculate_similarity(doc1, doc2)
resemblence

'7 / 17'

In [32]:
doc1 = "W3	W6	W3	W6	W7	W8	W9	W1	W4	W7	W3	W2	W1	W9"
doc2 = "W4	W7	W3	W6	W7	W8	W9	W1	W4	W7	W5	W3	W8	W9"

comb_s1 = sorted(generate_shingles(doc1))[:5]
comb_s2 = sorted(generate_shingles(doc2))[:5]

data = np.array([comb_s1, comb_s2]).T
df = pd.DataFrame(data, columns=["doc1", "doc2"])
df

,doc1,doc2
0,147,147
1,219,367
2,321,389
3,363,473
4,367,475


In [35]:
def calculate_similarity(doc1, doc2, num_lowest_shingles: int=None):
    
    if num_lowest_shingles:
        comb_s2 = sorted(generate_shingles(doc2))[:num_lowest_shingles]
        comb_s1 = sorted(generate_shingles(doc1))[:num_lowest_shingles]
    else:
        comb_s2 = generate_shingles(doc2)
        comb_s1 = generate_shingles(doc1)
        
    return f"{len(set(comb_s1).intersection(set(comb_s2)))} / {len(set(comb_s1).union(set(comb_s2)))}"

In [36]:
doc1 = "W3	W6	W3	W6	W7	W8	W9	W1	W4	W7	W3	W2	W1	W9"
doc2 = "W4	W7	W3	W6	W7	W8	W9	W1	W4	W7	W5	W3	W8	W9"

resemblence = calculate_similarity(doc1, doc2, num_lowest_shingles=5)
resemblence

'2 / 8'